In [1]:
import os

In [2]:
%pwd

'd:\\SAMITH\\Github\\Air-Quality-Health-Alert-System\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\SAMITH\\Github\\Air-Quality-Health-Alert-System'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class AlertConfig:
    root_dir: str
    sensitive_threshold: int
    general_threshold: int
    notification_method: str
    dashboard_refresh_interval_seconds: int = 60

In [6]:
from Air_Quality_Health_Alert_System.constants import *
from Air_Quality_Health_Alert_System.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])

    def get_alert_generation_config(self) -> AlertConfig:
        config = self.config.alert_generation
        alert_config = AlertConfig(
            root_dir=config.root_dir, 
            sensitive_threshold=config.sensitive_threshold,
            general_threshold=config.general_threshold,
            notification_method=config.notification_method,
            dashboard_refresh_interval_seconds=config.dashboard.refresh_interval_seconds
        )
        return alert_config

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import joblib
from sklearn.metrics import precision_score, recall_score, f1_score
from Air_Quality_Health_Alert_System import logger

In [ ]:
class AlertGenerator:
    def __init__(self, config: AlertConfig):
        self.config = config

    def check_threshold(self, predicted_aqi):
        sensitive_alert = predicted_aqi > self.config.sensitive_threshold
        general_alert = predicted_aqi > self.config.general_threshold
        messages = []
        if sensitive_alert:
            messages.append("Alert: AQI above sensitive threshold!")
        if general_alert:
            messages.append("Alert: AQI above general threshold!")
        logger.info(f"Thresholds checked: Sensitive Alert={sensitive_alert}, General Alert={general_alert}")
        return sensitive_alert, general_alert, messages

    def get_alerts_for_dashboard(self, predicted_aqi):
        _, _, messages = self.check_threshold(predicted_aqi)
        return messages

    def evaluate_alerts(self, predicted_aqi, actual_aqi):
        if hasattr(predicted_aqi, 'values'):
            predicted_aqi = predicted_aqi.values
        if hasattr(actual_aqi, 'values'):
            actual_aqi = actual_aqi.values
            
        y_true = (actual_aqi > self.config.sensitive_threshold).astype(int)
        y_pred = (predicted_aqi > self.config.sensitive_threshold).astype(int)
        logger.info("Alert evaluation metrics calculated successfully!")
        
        return {
            "precision": precision_score(y_true, y_pred, zero_division=0),
            "recall": recall_score(y_true, y_pred, zero_division=0),
            "f1": f1_score(y_true, y_pred, zero_division=0)
        }


In [10]:
try:
    config = ConfigurationManager()
    
    alert_generation_config = config.get_alert_generation_config()
    alert_generator = AlertGenerator(config=alert_generation_config)

    model_artifacts = joblib.load("artifacts/model_trainer/model.joblib")
    test_data = joblib.load("artifacts/model_trainer/test_data.joblib")

    model = model_artifacts['model']
    predicted_aqi = test_data['predictions']
    actual_aqi = test_data['y_test']

    
    results = alert_generator.evaluate_alerts(predicted_aqi, actual_aqi)

except Exception as e:
    raise e

[2025-08-22 11:51:11,953: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-22 11:51:11,958: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-22 11:51:11,962: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-08-22 11:51:11,964: INFO: common: created directory at: artifacts]
